In [1]:
import gymnasium as gym
import numpy as np
import scipy as sp

# Cliffwalking Environment

In [2]:
env = gym.make("CliffWalking-v0")
for k, v in vars(env).items():
    print(f"{k}: {v}")

_saved_kwargs: {'disable_render_order_enforcing': False}
env: <PassiveEnvChecker<CliffWalkingEnv<CliffWalking-v0>>>
_action_space: None
_observation_space: None
_metadata: None
_cached_spec: None
_has_reset: False
_disable_render_order_enforcing: False


In [3]:
env.action_space, env.observation_space

(Discrete(4), Discrete(48))

In [4]:
dynamics = env.env.env.P
dynamics

{0: {0: [(1.0, np.int64(0), -1, False)],
  1: [(1.0, np.int64(1), -1, False)],
  2: [(1.0, np.int64(12), -1, False)],
  3: [(1.0, np.int64(0), -1, False)]},
 1: {0: [(1.0, np.int64(1), -1, False)],
  1: [(1.0, np.int64(2), -1, False)],
  2: [(1.0, np.int64(13), -1, False)],
  3: [(1.0, np.int64(0), -1, False)]},
 2: {0: [(1.0, np.int64(2), -1, False)],
  1: [(1.0, np.int64(3), -1, False)],
  2: [(1.0, np.int64(14), -1, False)],
  3: [(1.0, np.int64(1), -1, False)]},
 3: {0: [(1.0, np.int64(3), -1, False)],
  1: [(1.0, np.int64(4), -1, False)],
  2: [(1.0, np.int64(15), -1, False)],
  3: [(1.0, np.int64(2), -1, False)]},
 4: {0: [(1.0, np.int64(4), -1, False)],
  1: [(1.0, np.int64(5), -1, False)],
  2: [(1.0, np.int64(16), -1, False)],
  3: [(1.0, np.int64(3), -1, False)]},
 5: {0: [(1.0, np.int64(5), -1, False)],
  1: [(1.0, np.int64(6), -1, False)],
  2: [(1.0, np.int64(17), -1, False)],
  3: [(1.0, np.int64(4), -1, False)]},
 6: {0: [(1.0, np.int64(6), -1, False)],
  1: [(1.0, np.in

In [5]:
dynamics[14][2] # each tuple has values (probability, next_state, reward, terminal)

[(1.0, np.int64(26), -1, False)]

The above result represents the following equation:
$$p(s_{26}, -1 \mid s_{14}, a_2) = 1$$
The lack of other entries in the list of `dynamics[14][2]` is enforced by the only outcome having probability 1.

In [6]:
dynamics[35] # The position right next to the completed state

{0: [(1.0, np.int64(23), -1, False)],
 1: [(1.0, np.int64(35), -1, False)],
 2: [(1.0, np.int64(47), -1, True)],
 3: [(1.0, np.int64(34), -1, False)]}

In [7]:
dynamics[44] # One of the failure states. It is technically impossible to reach this state, but if you happen to be here, moving anywhere but up will move you to the start.

{0: [(1.0, np.int64(32), -1, False)],
 1: [(1.0, np.int64(36), -100, False)],
 2: [(1.0, np.int64(36), -100, False)],
 3: [(1.0, np.int64(36), -100, False)]}

In [8]:
dynamics[32] # One of the states in which moving down results in a transition to a state with reward -100. We can see that the -100 reward action moves the agent to state 36 - the start.

{0: [(1.0, np.int64(20), -1, False)],
 1: [(1.0, np.int64(33), -1, False)],
 2: [(1.0, np.int64(36), -100, False)],
 3: [(1.0, np.int64(31), -1, False)]}

In [9]:
# Investigate the dictionary dynamics
actionable_states = set() # states you can perform an action from.
reachable_states = set() # states you can reach from transitioning there.
terminal_states = set()
probabilities = set()
rewards = set()

for state, actions in dynamics.items():
    actionable_states.add(state)
    for action, lst in actions.items():
        for prob, next_state, reward, terminal in lst:
            reachable_states.add(int(next_state))
            rewards.add(reward)
            probabilities.add(prob)
            if terminal:
                terminal_states.add(next_state)

In [10]:
actionable_states == set(range(48))

True

In [11]:
reachable_states == set(range(48)) - set(range(37, 47))

True

In [12]:
terminal_states

{np.int64(47)}

In [13]:
rewards

{-100, -1}

In [14]:
probabilities

{1.0}

So we will define the following sets:
- $R = \{-100, -1\}$,
- $S = \{0,1,\dots,47\}$,
- $A = \{0,1,2,3\}$

We now construct the dynamics matrix $P_{S_{t+1}, R_{t+1}\mid S_t, A_t}$

In [15]:
def encode_dynamics_row(next_state, reward, no_rewards, reward_map):
    return next_state * no_rewards + reward_map[reward]

def encode_dynamics_col(previous_state, no_actions, action):
    return previous_state * no_actions + action

no_states = 48
no_actions = 4
no_rewards = 2
reward_map = {-100: 0, -1: 1}
index_to_reward = {0: -100, 1: -1}

np.set_printoptions(threshold=np.inf)
dynamics_matrix = np.zeros((48, 2, 48, 4))
for state, actions in dynamics.items():
    for action, lst in actions.items():
        for prob, next_state, reward, terminal in lst:
            reward_idx = reward_map[reward]
            #dynamics_matrix[encode_dynamics_row(next_state, reward, no_rewards, reward_map), encode_dynamics_col(state, no_actions, action)] = prob
            dynamics_matrix[next_state, reward_idx, state, action] = prob

dynamics_matrix

array([[[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
 

In [16]:
P = env.env.env.P
dynamics_matrix1 = np.zeros((48,2,48,4))
for s in range(no_states):
    for a in range(no_actions):
        for (p, next_s, reward, _) in P[s][a]:
            r_index = 0 if reward == -100 else 1
            dynamics_matrix1[next_s, r_index, s, a] += p
np.linalg.norm(dynamics_matrix - dynamics_matrix1)

np.float64(0.0)

In [17]:
np.all(np.sum(dynamics_matrix, axis=(0,1)) == 1) # check that the sum of s and r (dimension 0) always results in 1.

np.True_

## Policy Evaluation
We're going to combine the initial Bellman equations from `feed_full.ipynb`. As a reminder, we list them here:
$$
\begin{align*}
v_\pi(s) &= \sum_{a\in A}\pi(a\mid s)q_\pi(s)\\
q_\pi(s, a) &= \sum_{s' \in S^+, r \in \mathbb{R}}p(s', r\mid s, a)[r + \gamma v_\pi(s')]
\end{align*}$$
When put together, we have
$$v_\pi(s) = \sum_{a\in A}\pi(a\mid s)\sum_{s' \in S^+, r \in \mathbb{R}}p(s', r\mid s, a)[r + \gamma v_\pi(s')]$$
Which simplifies nicely to:
$$v_\pi(s) = \sum_{r \in R}r \cdot \mathrm{Pr}[R_{t+1} = r\mid S_t = s] + \gamma \sum_{s' \in S} v_\pi(s')p(s'\mid s)$$
When vectorised, this looks like:
$$\vec{v}_\pi = P^\top_{R_{t + 1}\mid S_t}\vec{r} + \gamma P^\top_{S_{t+1}\mid S_t}\vec{v}_\pi$$
Rearranged:
$$(I - \gamma P^\top_{S_{t+1}\mid S_t})\vec{v}_\pi = P^\top_{R_{t + 1}\mid S_t}\vec{r}$$
For $q_\pi(s,a)$ we have:
$$\vec{q}_\pi = \vec{r} + \gamma P^\top_{S_{t+1}\mid S_t,A_t}\vec{v}_\pi$$

In [18]:
def evaluate_bellman(dynamics_matrix, policy, r, gamma=1, verbose=True):
    p_sp_given_s = np.sum(dynamics_matrix * policy[np.newaxis, np.newaxis, :, :], axis=(1,3)) # sum over r and a.
    p_sp_given_s[:, 47] = 0 # It is impossible to transition from a terminal state.

    p_r_given_s = np.sum(dynamics_matrix * policy[np.newaxis, np.newaxis, :, :], axis=(0,3)) # sum over s' and a
    
    lhs = np.eye(p_sp_given_s.shape[0]) - gamma * p_sp_given_s.transpose(1,0)

    rhs = p_r_given_s.transpose(1,0) @ r[:, np.newaxis] # This is r_pi.
    rhs[47] = 0 # Naturally, the reward at the terminal state should also be 0
    
    v = np.linalg.solve(lhs, rhs)[:, 0] # (S',)

    r_sa = np.sum(dynamics_matrix * r[np.newaxis, :, np.newaxis, np.newaxis], axis=(0,1)) # (S, A)

    p_sp_given_s_a = np.sum(dynamics_matrix, axis=1) # (S', S, A) [needs to become (S, A, S')]
    q = r_sa + gamma * p_sp_given_s_a.transpose(1, 2, 0) @ v
    return v, q

In [19]:
# Hyperparameters
gamma = 1

In [20]:
# First use uniform distribution for policy.
rewards = np.array([-100, -1])
policy = np.ones((no_states, no_actions)) / no_actions
policy[47] = 0
v, q = evaluate_bellman(dynamics_matrix, policy, rewards, gamma)
v

array([-65104.83759923, -65038.58278728, -64886.60152705, -64615.82843642,
       -64172.32462724, -63470.85252573, -62382.03961361, -60720.87142908,
       -58253.74184103, -54780.58582606, -50443.22723535, -46534.94093822,
       -65167.09241118, -65120.30923556, -65001.39335745, -64784.55915496,
       -64426.29291957, -63854.19333635, -62950.39488602, -61522.83283259,
       -59255.76826794, -55640.7884018 , -50010.15494176, -42622.6546411 ,
       -65272.13039875, -65270.16838633, -65210.10351224, -65090.72190641,
       -64890.09455971, -64565.23301409, -64038.51376154, -63160.29674731,
       -61601.70999636, -58512.64457143, -51329.94948881, -31318.8680433 ,
       -65375.13039875, -65399.38989565, -65409.12367712, -65379.27827567,
       -65329.12143899, -65247.90605259, -65116.22623945, -64896.67198589,
       -64507.02529816, -63734.75894192, -45570.55257158,      0.        ])

Verifying $v_\pi$ and $q_\pi$:

In [21]:
use_policy = policy.copy()
use_policy[47] = 0
v_test = np.sum(use_policy * q, axis=1)
np.all(np.isclose(v_test - v, 0))

np.True_

In [22]:
# Now we will use the optimal policy
actions = np.ones(no_states)
actions[36] = 0
actions[11::12] = 2
actions

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.])

In [23]:
policy = np.eye(no_actions)[actions.astype(int)] # Clever one-hot encoding of policy
policy

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],


In [24]:
v_optimal, q_optimal = evaluate_bellman(dynamics_matrix, policy, rewards)
v_optimal

array([ -14.,  -13.,  -12.,  -11.,  -10.,   -9.,   -8.,   -7.,   -6.,
         -5.,   -4.,   -3.,  -13.,  -12.,  -11.,  -10.,   -9.,   -8.,
         -7.,   -6.,   -5.,   -4.,   -3.,   -2.,  -12.,  -11.,  -10.,
         -9.,   -8.,   -7.,   -6.,   -5.,   -4.,   -3.,   -2.,   -1.,
        -13., -113., -113., -113., -113., -113., -113., -113., -113.,
       -113.,   -1.,    0.])

# Solving for Optimal Values
We want to choose a $v(s)$ that will minimise
$$\sum_{s \in S}v(s)$$
such that for all $s \in S$ and $a\in A$:
$$v(s) - \gamma\sum_{s'\in S^+, r\in R}p(s', r\mid s, a)v(s') \geq \sum_{s' \in S^+, r\in R}r\cdot p(s', r\mid s, a)$$
which simplifies to
$$v(s) - \gamma\sum_{s' \in S^+}p(s'\mid s, a)v(s') \geq \sum_{r\in R}r\cdot p(r\mid s, a)$$
i.e:
$$(I - \gamma P_{S_{t+1}\mid S_t, A_t}^\top)\vec{v} \geq P_{R_{t+1}\mid S_t, A_t}\vec{r}$$

In [48]:
def optimal_bellman(dynamics_matrix, rewards, gamma=1., verbose=True):
    p = np.sum(dynamics_matrix, axis=1) # (s', s, a)
    p[:, 47, :] = 0
    p = p.transpose(1, 2, 0)

    r = np.sum(dynamics_matrix, axis=0).transpose(1,2,0) @ rewards
    r[47] = 0
    
    c = np.ones(48)
    a_ub = gamma * p.reshape(-1, 48) - np.repeat(np.eye(48), 4, axis=0)
    b_ub = -r.reshape(-1)
    a_eq = np.zeros((0, 48))
    b_eq = np.zeros(0)
    bounds = [(None, None),] * 48
    
    res = sp.optimize.linprog(c, a_ub, b_ub, bounds=bounds, method='highs')
    v = res.x
    q = r + gamma * np.dot(p, v)
    if verbose:
        print('optimal state values =', v)
        print('optimal action values =', q)
    return v, q
optimal_state_values, optimal_action_values = optimal_bellman(dynamics_matrix, rewards)

optimal state values = [-14. -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3. -13. -12.
 -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3.  -2. -12. -11. -10.  -9.
  -8.  -7.  -6.  -5.  -4.  -3.  -2.  -1. -13. -12. -11. -10.  -9.  -8.
  -7.  -6.  -5.  -4.  -1.  -0.]
optimal action values = [[ -15.  -14.  -14.  -15.]
 [ -14.  -13.  -13.  -15.]
 [ -13.  -12.  -12.  -14.]
 [ -12.  -11.  -11.  -13.]
 [ -11.  -10.  -10.  -12.]
 [ -10.   -9.   -9.  -11.]
 [  -9.   -8.   -8.  -10.]
 [  -8.   -7.   -7.   -9.]
 [  -7.   -6.   -6.   -8.]
 [  -6.   -5.   -5.   -7.]
 [  -5.   -4.   -4.   -6.]
 [  -4.   -4.   -3.   -5.]
 [ -15.  -13.  -13.  -14.]
 [ -14.  -12.  -12.  -14.]
 [ -13.  -11.  -11.  -13.]
 [ -12.  -10.  -10.  -12.]
 [ -11.   -9.   -9.  -11.]
 [ -10.   -8.   -8.  -10.]
 [  -9.   -7.   -7.   -9.]
 [  -8.   -6.   -6.   -8.]
 [  -7.   -5.   -5.   -7.]
 [  -6.   -4.   -4.   -6.]
 [  -5.   -3.   -3.   -5.]
 [  -4.   -3.   -2.   -4.]
 [ -14.  -12.  -14.  -13.]
 [ -13.  -11. -113.  -13.]
 [ -12